In [2]:
import numpy as np
import pandas as pd
import requests
import random

{'adult': False, 'backdrop_path': '/2DbSpBgjHASSOXe5eHC9qVtif0F.jpg', 'belongs_to_collection': {'id': 705370, 'name': 'Beauty Salon: Special Service Series', 'poster_path': None, 'backdrop_path': None}, 'budget': 0, 'genres': [{'id': 10749, 'name': 'Romance'}, {'id': 18, 'name': 'Drama'}], 'homepage': '', 'id': 452251, 'imdb_id': '', 'original_language': 'ko', 'original_title': '미용실 : 특별한 서비스', 'overview': 'A legendary beauty salon that is sure to become a regular once you go!”Sir, would you like to have a very hot service?”In order to overcome the financial difficulties of the beauty parlor,It begins to attract the male guests with the special technique that makes it from head to toe.Kurt, Firm, and ‘it’ to open the hair salon is good!', 'popularity': 138.219, 'poster_path': '/z3Vse8jRnmKJyZFK6VSw1G6jHnY.jpg', 'production_companies': [], 'production_countries': [{'iso_3166_1': 'KR', 'name': 'South Korea'}], 'release_date': '2016-12-29', 'revenue': 0, 'runtime': 81, 'spoken_languages': [{'english_name': 'Korean', 'iso_639_1': 'ko', 'name': '한국어/조선말'}], 'status': 'Released', 'tagline': '', 'title': 'Beauty Salon: Special Service', 'video': False, 'vote_average': 6.2, 'vote_count': 11}

In [4]:
MAX = 10

API_KEY = "48e0bf10bd721602a930738080e4daf4"
url = f"https://api.themoviedb.org/3/movie/{{}}?api_key={API_KEY}&append_to_response=budget,revenue"

all_movies = []

page_number = 1
while page_number < MAX:
    random_number = random.random()
    random_page_number = random.randint(1, MAX)
    response = requests.get(f"https://api.themoviedb.org/3/discover/movie?api_key={API_KEY}&sort_by=vote_count.desc&random={random_number}&page={random_page_number}")
    if response.status_code == 200:
        data = response.json()
        for movie in data['results']:
            movie_id = movie['id']
            movie_response = requests.get(url.format(movie_id))
            if movie_response.status_code == 200:
                movie_data = movie_response.json()
                cast_response = requests.get(f"https://api.themoviedb.org/3/movie/{movie_id}/credits?api_key={API_KEY}")
                if cast_response.status_code == 200:
                    cast_data = cast_response.json()
                    budget = movie_data['budget']
                    revenue = movie_data['revenue']
                    if budget == 0 or revenue == 0:
                        continue  # exclude movies with budget or revenue 0
                    release_date = movie_data['release_date']
                    if release_date:
                        release_decade = release_date[:3] + '0s'
                    else:
                        release_decade = None
                    all_movies.append({
                        'Title': movie_data['title'],
                        'Genres': [genre['name'] for genre in movie_data['genres']],
                        'Cast': [actor['name'] for actor in cast_data['cast']][:5],
                        'Budget': budget,
                        'Revenue': revenue,
                        'Popularity': movie_data['popularity'],
                        'Vote_average': movie_data['vote_average'],
                        'Release_decade': release_decade,
                    })
                else:
                    print(f"Error {cast_response.status_code}: {cast_response.reason}")
                    break
            else:
                print(f"Error {movie_response.status_code}: {movie_response.reason}")
                break
        total_pages = data['total_pages']
        if page_number >= total_pages:
            break
        page_number += 1
    else:
        print(f"Error {response.status_code}: {response.reason}")
        break

df = pd.DataFrame(all_movies)
df.head()

,Title,Genres,Cast,Budget,Revenue,Popularity,Vote_average,Release_decade
0,Shutter Island,"[Drama, Thriller, Mystery]","[Leonardo DiCaprio, Mark Ruffalo, Ben Kingsley...",80000000,294800000,46.067,8.197,2010s
1,The Lord of the Rings: The Return of the King,"[Adventure, Fantasy, Action]","[Elijah Wood, Ian McKellen, Liv Tyler, Viggo M...",94000000,1118888979,91.431,8.474,2000s
2,The Wolf of Wall Street,"[Crime, Drama, Comedy]","[Leonardo DiCaprio, Jonah Hill, Margot Robbie,...",100000000,392000000,127.416,8.037,2010s
3,Avengers: Age of Ultron,"[Action, Adventure, Science Fiction]","[Robert Downey Jr., Chris Hemsworth, Mark Ruff...",365000000,1405403694,136.706,7.281,2010s
4,Captain America: Civil War,"[Adventure, Action, Science Fiction]","[Chris Evans, Robert Downey Jr., Scarlett Joha...",250000000,1153337496,101.111,7.440,2010s
